In [1]:
import numpy as np
import cv2 
import mediapipe as mp
import pandas as pd
import glob
mp_draw=mp.solutions.drawing_utils
mp_pose=mp.solutions.pose
lst_sample=[]

In [2]:
# path=glob.glob('chute07/*')

In [3]:
# path

In [4]:
def detection(img,model):
    img=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    img.flags.writeable=False
    results=model.process(img)
    img.flags.writeable=True
    img=cv2.cvtColor(img,cv2.COLOR_RGB2BGR)
    return img,results


In [5]:
def lay_mau(landmark,frame):
    a=[]
    lst_x=[]
    lst_y=[]
    for idx,lmk in enumerate(landmark):
        if idx in [x for x in range(1,11)]:
            continue
        elif idx in [x for x in range(17,23)]:
            continue
        # y=landmark[0].y
        # cv2.putText(frame,'Cor:{}'.format(y),(50,50),cv2.FONT_HERSHEY_COMPLEX,1,(255,0,0),1,cv2.LINE_AA)
        if idx in [0,12,11,23,24,27,28]:
            lst_x.append(lmk.x)
            lst_y.append(lmk.y)
        a.append(lmk.x)
        a.append(lmk.y)
        a.append(lmk.z)
    return a,lst_x,lst_y

In [6]:
import os
path=glob.glob('VIDEO_TEST/*.mp4')
path=sorted(path,key=os.path.getmtime)
path.remove(path[-1])
path


['VIDEO_TEST\\VIDEO1.mp4',
 'VIDEO_TEST\\VIDEO2.mp4',
 'VIDEO_TEST\\VIDEO3.mp4',
 'VIDEO_TEST\\VIDEO4.mp4',
 'VIDEO_TEST\\VIDEO5.mp4',
 'VIDEO_TEST\\VIDEO6.mp4',
 'VIDEO_TEST\\VIDEO7.mp4',
 'VIDEO_TEST\\VIDEO8.mp4',
 'VIDEO_TEST\\VIDEO9.mp4',
 'VIDEO_TEST\\VIDEO10.mp4',
 'VIDEO_TEST\\VIDEO11.mp4',
 'VIDEO_TEST\\VIDEO12.mp4',
 'VIDEO_TEST\\VIDEO13.mp4',
 'VIDEO_TEST\\VIDEO14.mp4',
 'VIDEO_TEST\\VIDEO15.mp4',
 'VIDEO_TEST\\VIDEO16.mp4',
 'VIDEO_TEST\\VIDEO17.mp4',
 'VIDEO_TEST\\VIDEO18.mp4',
 'VIDEO_TEST\\VIDEO19.mp4',
 'VIDEO_TEST\\VIDEO20.mp4',
 'VIDEO_TEST\\VIDEO21.mp4']

In [7]:
def xulylandmark(lst_x,lst_y,frame):
    waistx=(lst_x[3]+lst_x[4])/2*frame.shape[1]
    waisty=(lst_y[3]+lst_y[4])/2*frame.shape[0]
    xcenter=(lst_x[0]*frame.shape[1]+waistx)/2
    ycenter=(lst_y[0]*frame.shape[0]+waisty)/2
    footx=(lst_x[-2]+lst_x[-1])/2*frame.shape[1]
    widh=abs(xcenter-footx)
    footy=(lst_y[-2]+lst_y[-1])/2*frame.shape[0]
    heit=abs(ycenter-footy)
    return widh,heit
    

In [8]:
# cap=cv2.VideoCapture('VIDEO_TEST/2.mp4')#path chon video 
frame_num=30
lst_status=[]
lst_sample=[]#comma khi lay mau khac ngoai fall
with mp_pose.Pose(min_detection_confidence=0.5,min_tracking_confidence=0.5,model_complexity=1) as pose:
    for i in path:
        # cap=cv2.VideoCapture('C:/Users/PhuTuan/Downloads/Video/walking1.mp4')
        cap = cv2.VideoCapture(i)
        while cap.isOpened():
            ret,frame=cap.read()
            if ret:
                img,results=detection(frame,pose)
                mp_draw.draw_landmarks(img,results.pose_landmarks,mp_pose.POSE_CONNECTIONS)
                try:
                    landmark=results.pose_landmarks.landmark
                    # lst_sample.append(lay_mau(landmark,img))
                    sample,lst_x,lst_y=lay_mau(landmark,img)
                    # print(lst_x,lst_y)
                    width,height=xulylandmark(lst_x,lst_y,img)
                    if width>height:
                        frame_num-=1
                        lst_sample.append(sample)
                        print(len(lst_status))
                        if frame_num==0:
                            lst_status.append(lst_sample)
                            lst_sample=[]
                            frame_num=30
                            print(len(lst_status))

                except:
                    pass
                # mp_draw.draw_landmarks(img,results.pose_landmarks,mp_pose.POSE_CONNECTIONS)
                # lst_sample.append(samples)
                # if cv2.waitKey(20)&0xFF==ord('w'):
                #     frame_num-=1
                #     lst_sample.append(lay_mau(landmark,img))
                #     print(len(lst_sample))
                #     if frame_num==0:
                #         lst_status.append(lst_sample)
                #         lst_sample=[]
                #         frame_num=30
                #         print(len(lst_status))
                if cv2.waitKey(20)& 0xFF==ord('q'):
                    break
                cv2.imshow('Pose',img)
            else:
                break
        cap.release()
# df =pd.DataFrame(lst_sample)
# df.to_csv('Sample.txt')
cv2.destroyAllWindows()

0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5


In [9]:
len((lst_status))

5

In [10]:
for i in range(len(lst_status)):
    df=pd.DataFrame(lst_status[i])
    name='Sample'+str(i+119)
    df.to_csv('SAMPLE_w/{}.txt'.format(name))